# ID2214 Assignment 3 Group no. [7]
### Project members: 
[Cecilia Battinelly, cbat@kth.se]
[Valgerdur Tryggvadottir, vtry@kth.se]
[Parastu Rahgozar, parastu@kth.se]
[Zhiwu Dong, zhiwu@kth.se

### Declaration
By submitting this solution, it is hereby declared that all individuals listed above have contributed to the solution, either with code that appear in the final solution below, or with code that has been evaluated and compared to the final solution, but for some reason has been excluded. It is also declared that all project members fully understand all parts of the final solution and can explain it upon request.

It is furthermore declared that the code below is a contribution by the project members only, and specifically that no part of the solution has been copied from any other source (except for lecture slides at the course ID2214) and no part of the solution has been provided by someone not listed as project member above.

It is furthermore declared that it has been understood that no other library/package than the Python 3 standard library, NumPy, pandas and time may be used in the solution for this assignment.

### Instructions
All assignments starting with number 1 below are mandatory. Satisfactory solutions
will give 1 point (in total). If they in addition are good (all parts work more or less 
as they should), completed on time (submitted before the deadline in Canvas) and according
to the instructions, together with satisfactory solutions of assignments starting with 
number 2 below, then the assignment will receive 2 points (in total).

It is highly recommended that you do not develop the code directly within the notebook
but that you copy the comments and test cases to your regular development environment
and only when everything works as expected, that you paste your functions into this
notebook, do a final testing (all cells should succeed) and submit the whole notebook 
(a single file) in Canvas (do not forget to fill in your group number and names above).


## Load NumPy, pandas and time

In [1]:
import numpy as np
import pandas as pd
import time


## Reused functions from Assignment 1

In [2]:
# Copy and paste functions from Assignment 1 here that you need for this assignment

# CREATE BINS FUNCTION
def create_bins(dff,nobins=10,bintype="equal-width"):
    d = {}
    df = dff.copy()
    if bintype=="equal-width":
        for elem in df.columns:
            if elem!='ID' and elem!='CLASS':
                values = df[elem].values
                res, bins = pd.cut(values,nobins,retbins=True,labels=False)
                bins=list(bins)
                bins[0] = -np.inf
                bins[-1] = np.inf
                d[elem] = bins #mapping
                df[elem]= res
                df[elem] = df[elem].astype("category")
                df[elem].cat.categories
                lab = list(range(0,nobins))
                df[elem] = df[elem].cat.set_categories(lab)
    else:
        for elem in df.columns:
            if elem!='ID' and elem!='CLASS':
                values = df[elem].values
                res, bins = pd.qcut(values,nobins,retbins=True,labels=False, duplicates='drop')
                bins = list(bins)
                bins[0] = -np.inf
                bins[-1]= np.inf
                d[elem] =  bins #mapping
                df[elem] = res
                df[elem] = df[elem].astype("category")
                df[elem].cat.categories
                lab = list(range(0,nobins))
                df[elem] = df[elem].cat.set_categories(lab)


    return df, d


# CREATE IMPUTATION FUNCTION
def create_imputation(df1):
    df = df1.copy()
    d = {}
    # Go through all the columns whose name is different from ID & CLASS
    # First we analyze numerical columns and if all the values are missing
    # we fill them with 0, otherwise we fill the missing values with the mean.
    # We append to the dictionary the mean value of the column referring to the column
    # name as key.

    # Later(else) we go through object and category columns. If all values are missing
    # we fill them with "" for object column, or first element for category column
    # otherwise we fill with the first element of mode array.
    # We append to the dictionary the mode value [0] of the column referring to the column
    # name as key.
    
    for elem in df.columns:
        if elem!='ID' and elem!='CLASS':
            if df[elem].dtype == "float64" or df[elem].dtype == "int64":
                values = df[elem].values     
                if np.all(np.isnan(values)):
                    df[elem].fillna(0, inplace = True)
                else:
                      df[elem].fillna(df[elem].mean(), inplace = True)
                    

                value = df[elem].mean() 
                d[elem] = value
            else:
                values = df[elem].values
                df[elem].fillna(df[elem].mode()[0], inplace = True)

                value2 = df[elem].mode()[0]
                d[elem] = value2

    return df, d


# APPLY IMPUTATION
def apply_imputation(df1,imputation):
    df = df1.copy()
    
    # Go through all columns whose name is not ID or CLASS and fill the missing
    # values reffering to the imputation dictionary, i.e, given the column name, we fill
    # the missing value using the valueZ in dictionary (elem: valueZ).

    for elem in df1.columns:
        if elem!='ID' and elem!='CLASS':
            tup_elem = imputation[elem]
            df[elem].fillna(tup_elem, inplace = True)

    return df



# APPLY BINS FUNCTION
def apply_bins(dff,binning):
    df = dff.copy()
    for elem in df.columns:
        if elem!='ID' and elem!='CLASS':
                values = df[elem].values
                bins = binning[elem]
                res = pd.cut(values,bins,labels=False)
                df[elem] = res
                df[elem] = df[elem].astype("category")
                df[elem].cat.categories
    return df


# ACCURACY FUNCTION
def accuracy(df, correctlabels):
    dff = df.copy()
    if len(correctlabels)!= len(dff.index):
         print("Error, mismatching dimensions!")
         return None
    maximum = dff.idxmax(axis=1)
    num = 0
    
    for i in range(len(maximum)):
        if maximum[i] == correctlabels[i]:
            num += 1
    den = len(maximum)
    
    frac = num / den

    return frac


# BRIER SCORE FUNCTION
def brier_score(df,correctlabels):
    # Create an empty list where all partial sums will be stored
    score = []
    
    for i in df.index:
        # Create partial sum for each row
        summ = 0
        row = np.array(df.loc[i])
        
        o = np.zeros(len(row))
        indx = np.where(df.columns==correctlabels[i])[0]
        o[indx] = 1
        for i in range(len(row)):
            summ += (row[i] - o[i])**2
        score.append(summ)

    brier = sum(score)/len(df.index)
    
    return brier


# AUC FUNCTION
def auc(df,correctlabels):
    #Create a vector where to store partial AUCs for each class (i.e. column) 
    AUCT = [] 
    for col in df.columns:
         # For each class create a dictionary to store {score: tp, fp}
        d = {}
        values = df[col].values
        # Create array of scores values in decreasing order
        val = sorted(values, reverse = True)  
        # For each element in val iterate and set the threshold equal to the 
        # current value and label the instances accordingly to this 
        for elem in val:
            # Current score = threshold = each value in the column iteratively
            s = elem 
            tp = 0
            fp = 0
            # For each score in the unordered list of scores check if the label is correct
            # If so, increase by 1 tp; o/w increase by 1 fp
            for j in range(len(values)):
                if values[j] == s:
                    label = col
                    if label == correctlabels[j]:
                        tp += 1
                    else:
                        fp += 1
            # Append the values of fp, tp to the corresponding score
            d[s] = [tp,fp]
        # Implementation of AUC area algorithm for each class
        Tot_tp = 0
        Tot_fp = 0
        for elem in d:
            Tot_tp += d[elem][0]
            Tot_fp += d[elem][1]
        AUC = 0
        Cov_tp = 0
        for elem in d:
            fp = d[elem][1]
            tp = d[elem][0]
            if fp == 0:
                Cov_tp += tp
            elif tp == 0:
                AUC += (Cov_tp/Tot_tp)*(fp/Tot_fp)
            else:
                AUC += (Cov_tp/Tot_tp)*(fp/Tot_fp)+(tp/Tot_tp)*(fp/Tot_fp)/2
                Cov_tp += tp
        AUCT.append(AUC)
    
    fr = []
    for col in df.columns:
        freq = 0
        for val in correctlabels:
            if val == col:
                freq += 1
        freq = freq /len(correctlabels)
        fr.append(freq)
    
    # Return weighted auc = return scalar product AUCT & frequency vector
    return np.dot(fr,AUCT)



## 1. Define the class DecisionTree

In [3]:
# Define the class DecisionTree with three functions __init__, fit and predict (after the comments):
#
# Input to __init__: 
# self: the object itself
#
# Output from __init__:
# nothing
# 
# This function does not return anything but just initializes the following attributes of the object (self) to None:
# binning, imputatiom, labels, model
#
# Input to fit:
# self: the object itself
# df: a dataframe (where the column names "CLASS" and "ID" have special meaning)
# nobins: no. of bins (default = 10)
# bintype: either "equal-width" (default) or "equal-size"
# min_samples_split: no. of instances required to allow a split (default = 5)
#
# Output from fit:
# nothing
#
# The result of applying this function should be:
#
# self.binning should be a discretization mapping (see Assignment 1) from df
# self.imputation should be an imputation mapping (see Assignment 1) from df
# self.labels should be the categories of the "CLASS" column of df, set to be of type "category" 
# self.model should be a decision tree (for details, see lecture slides), where the leafs return class probabilities
# Note that the function does not return anything but just assigns values to the attributes of the object.
#
# Hint 1: First find the available features (excluding "CLASS" and "ID"), then find the class counts, e.g., using 
#         groupby, and calculate the default class probabilities (relative frequencies of the class labels)
# Hint 2: Define a function, e.g., called divide_and_conquer, that takes the above as input together with df 
#         and min_samples_split, and also a nodeno (starting with 0) to keep track of the generated nodes in the tree
# Hint 3: You may represent the tree under construction as a list of nodes (tuples), on the form:
#         (nodeno,"leaf",class_probabilities): corresponding to a leaf node where class_probabilities is a vector
#                                              with the relative class frequencies (ordered according to self.labels)
#         (nodeno,feature,node_dict): corresponding to an internal (non-leaf) node where node_dict is a mapping from
#                                     the possible values of feature to child nodes (their nodenos)
# Hint 4: You may evaluate each feature by a function information_content, which takes the group sizes
#         for each possible value of the feature together with the class counts of each group as input
# Hint 5: The best feature found (with lowest resulting information content) will be used to split the training
#         instances, and each sub-group is used for generating a sub-tree (recursively by divide_and_conquer,
#         see lecture slides for details)
# Hint 6: The list of nodes output by divide_and_conquer may finally be converted to an array, where each nodeno in the 
#         tuples corresponds to an index of the array 
#
# Input to predict:
# self: the object itself
# df: a dataframe
# 
# Output from predict:
# predictions: a dataframe with class labels as column names and the rows corresponding to
#              predictions with estimated class probabilities for each row in df, where the class probabilities
#              are the relative class frequencies in the leaves of the decision tree into which the instances in
#              df fall
#
# Hint 1: Drop any "CLASS" and "ID" columns first and then apply imputation and binning
# Hint 2: Iterate over the rows calling some sub-function, e.g., make_prediction(nodeno,row), which for a test row
#         finds a leaf node from which class probabilities are obtained
# Hint 3: This sub-function may recursively traverse the tree (represented by an array), starting with the nodeno
#         that corresponds to the root

class DecisionTree:
    
    def __init__(self):
        self.binning = None
        self.imputation = None
        self.labels = None
        self.model = None


        
    def fit(self, df, nobins=10, bintype="equal-width", min_samples_split = 5):
        
        # self.imputation should be an imputation mapping from df
        df1, imp_map = create_imputation(df) 
        self.imputation = imp_map
        
        # self.binning should be a discretization mapping from df
        df2, disc_map = create_bins(df1, nobins, bintype)
        self.binning = disc_map
        
        # self.labels should be the categories of the "CLASS" column of df, set to be of type "category" 
        classes = df["CLASS"].astype("category")  # gives the column "CLASS", each column in DataFrame is of type Series
        self.labels = list(set(classes))      
        

        # Hint 1: First find the available features (excluding "CLASS" and "ID"), then find the class counts, e.g., using 
        #         groupby, and calculate the default class probabilities (relative frequencies of the class labels)
        
        # Available features = coulmn names except "CLASS" and "ID"
        df_features = df2.drop(["ID", "CLASS"], axis = 1)  
        features = list(df_features.columns.values)
        
        # relative frequencies  
        classprior = dict(df2.groupby("CLASS").size()/len(df2))    
        
        
        
        
        # Hint: Define a function, e.g., called divide_and_conquer, that takes the above as input together with df 
        #         and min_samples_split, and also a nodeno (starting with 0) to keep track of the generated nodes in the tree
        def divide_and_conquer(df, feature, nodeno, min_samples_split, class_prob=classprior):
    
                T = []
                
                # ----- STEP 1 -----:  There are three stopping criterias, we start by checking them    

                #If #instances is lower than the minimum number of instances needed to do a split => return the default probabilities
                if len(df) < min_samples_split: 
                    new_class_prob = dict(df.groupby("CLASS").size()/len(df))
                    node = (nodeno,"leaf", class_prob) 
                    T.append(node) # Add the tuple to the final tree & return it
                    return T
                
                #If there is only 1 class, i.e. all instances have same class label
                elif len(list(set(df["CLASS"])))==1: 
                    new_class_prob = dict(df.groupby("CLASS").size()/len(df)) #always 1
                    node = (nodeno, "leaf", new_class_prob)
                    T.append(node)
                    return T
                
                # If there are no more features to use to do split
                elif len(feature) == 0:
                    new_class_prob = dict(df.groupby("CLASS").size()/len(df))
                    node = (nodeno,"leaf", new_class_prob)
                    T.append(node)
                    return T
                
                
                
                # Else, we are not in a leaf and we should call the function again
                else:
                    # ----- STEP 2 -----:  Define the information_content function
                    # Hint: You may evaluate EACH feature by a function information_content, which takes the group sizes
                    #       for each possible value of the feature together with the class counts of each group as input
                    def information_content(group_size, class_count_group):
                                            
                        # Want the information content to be as low as possible (so this is just the second term
                        # of the information gain fromula)
                        # Information content = sum over all groups of feature[(group_size/total)*entropy(group)]
                        # Entropy(group) = sum over possible classes of that group (-pi log (pi))  => use class_count_group
                        
                        IC = 0  #information content initialized
                        total = sum(group_size)
                        for group in dict(group_size).keys():  # the groups are the values of bins
                            entropy = 0 #for each group, so have to initialize here as 0 
                            if group_size[group] != 0:  #because if group size is zero it will not appear in class_count_group
                                for amount in class_count_group[group]:
                                    p = amount/group_size[group]
                                    entropy -= p*np.log2(p)
    
                                IC += (group_size[group]/total)*entropy
                        
                        
                        return IC  
                
                
                    # ----- STEP 3 -----:  find where to split
                    # Now we can use the information_content function
                    group_size = 0 
                    IC_list = []
                    
                    # Call the function for each feature
                    for i in feature:  
                        group_size = df.groupby(i).size()
                        class_count_group = df.groupby([i, "CLASS"]).size() 
                        IC = information_content(group_size, class_count_group)
                        IC_list.append(IC)
                   
                    # Hint: The best feature found (with lowest resulting information content) will be used to split the training
                    #         instances, and each sub-group is used for generating a sub-tree (recursively by divide_and_conquer,
                    #         see lecture slides for details)
                    min_index = np.argmin(IC_list)  # This is the index of the feature we want to split
                    f = feature[min_index] #the feature where to split
                    feature.remove(f) #remove the splitting feature (because we are only allowed to use each one once to split)
                    
                    
                    # ----- STEP 4 -----:  Split and call the the function recursively
    
                    # Create a dictionary to store child node for each value of the column f
                    nodedict = {} 
                    classCountI = dict(df.groupby("CLASS").size()/len(df)) #Current default priors for class
                    for val, df_val in df.groupby(f):
                        newnodeno = nodeno + 1 # Here we increase the node on a level of depth (so nodeno is the depth of the tree)
                        T_val = divide_and_conquer(df_val, feature, newnodeno, min_samples_split, class_prob = classCountI)
                        nodedict[val] = T_val # Will create a nested dictionary
                
                    # define nodes and append to T
                    node = (nodeno, f ,nodedict)  
                    T.append(node)
                    
                    return T

        
        # nodeno should first be 0 
        nodeno = 0
        self.model = divide_and_conquer(df2, features, nodeno, min_samples_split, classprior)
        
        
       
    
    def predict(self, df):
        
        # Hint 1: Drop any "CLASS" and "ID" columns first and then apply imputation and binning
        df_copy = df.drop(["CLASS", "ID"], axis = 1)
        df_imputation = apply_imputation(df_copy, self.imputation)
        df_binning = apply_bins(df_imputation, self.binning)
        
        # Get the list of features names for making predictions 
        features = list(df_binning.columns.values)
        
        
        # The following func makes a prediction for an instance (i.e. a row of the test df)
        # using the tree, i.e. self.model and the features to easily access the tree
        def make_prediction(tree, row, features):
            
            col = tree[0][1] # Splitting feature 
            feat = np.array(features) 
            col_num = np.where(feat==col) # Get the index of the splitting feature to access
            value = row[col_num]  # its value in the row
            
            # Now we want to find this values among the childnodes of the splitting feature
            subtree = list(tree[0][2].items()) 
            # We access to subtree, i.e. node_dict which is storing for each value of splitting feature its subsubtrees
            for key, elem in subtree:
                if key == value and "leaf" in elem[0][1]:
                    # This means we are ended up in a leaf and we convert the 
                    # dictionary of prob into vector
                    pr = np.zeros(len(self.labels))
                    classprob = elem[0][2].items()
                    labels = np.array(self.labels)
                    
                    # Conversion from dictionary into array for probabilities
                    for clas,prob in classprob:
                         idx = np.where(labels==clas)
                         pr[idx] = prob

                    return pr

                elif key == value and "leaf" not in elem[0][1]:
                    # This means that the value does not correspond to a lef
                    # We need to go further down the tree to investigate and we call the func
                    # again on the subtree
                    return make_prediction(elem, row, features)

                    
                
        prediction_list = []
        for i in range(len(df_binning.values)):
            row = df_binning.values[i]
            prob = make_prediction(self.model, row, features) #call the function make_prediction
            prediction_list.append(prob)
        
        predictions = pd.DataFrame(prediction_list, columns = self.labels)  

        return predictions






In [4]:
# Test your code (leave this part unchanged, except for if auc is undefined)

glass_train_df = pd.read_csv("glass_train.txt")

glass_test_df = pd.read_csv("glass_test.txt")

tree_model = DecisionTree()

test_labels = glass_test_df["CLASS"]

nobins_values = [5,10]
bintype_values = ["equal-width","equal-size"]
min_samples_split_values = [3,5,10]
parameters = [(nobins,bintype,min_samples_split) for nobins in nobins_values for bintype in bintype_values 
              for min_samples_split in min_samples_split_values]

results = np.empty((len(parameters),3))

for i in range(len(parameters)):
    t0 = time.perf_counter()
    tree_model.fit(glass_train_df,nobins=parameters[i][0],bintype=parameters[i][1],min_samples_split=parameters[i][2])
    print("Training time {0}: {1:.2f} s.".format(parameters[i],time.perf_counter()-t0))
    t0 = time.perf_counter()
    predictions = tree_model.predict(glass_test_df)
    print("Testing time {0}: {1:.2f} s.".format(parameters[i],time.perf_counter()-t0))
    results[i] = [accuracy(predictions,test_labels),brier_score(predictions,test_labels),
                  auc(predictions,test_labels)] # Assuming that you have defined auc - remove otherwise

results = pd.DataFrame(results,index=pd.MultiIndex.from_product([nobins_values,bintype_values,min_samples_split_values]),
                       columns=["Accuracy","Brier score","AUC"])

results



Training time (5, 'equal-width', 3): 0.26 s.
Testing time (5, 'equal-width', 3): 0.12 s.
Training time (5, 'equal-width', 5): 0.26 s.
Testing time (5, 'equal-width', 5): 0.14 s.
Training time (5, 'equal-width', 10): 0.20 s.
Testing time (5, 'equal-width', 10): 0.12 s.
Training time (5, 'equal-size', 3): 0.21 s.
Testing time (5, 'equal-size', 3): 0.09 s.
Training time (5, 'equal-size', 5): 0.22 s.
Testing time (5, 'equal-size', 5): 0.10 s.
Training time (5, 'equal-size', 10): 0.22 s.
Testing time (5, 'equal-size', 10): 0.09 s.
Training time (10, 'equal-width', 3): 0.29 s.
Testing time (10, 'equal-width', 3): 0.08 s.
Training time (10, 'equal-width', 5): 0.30 s.
Testing time (10, 'equal-width', 5): 0.08 s.
Training time (10, 'equal-width', 10): 0.43 s.
Testing time (10, 'equal-width', 10): 0.10 s.
Training time (10, 'equal-size', 3): 0.30 s.
Testing time (10, 'equal-size', 3): 0.10 s.
Training time (10, 'equal-size', 5): 0.30 s.
Testing time (10, 'equal-size', 5): 0.10 s.
Training time (

Accuracy  Brier score       AUC
5  equal-width 3   0.485981     0.563166  0.668203
               5   0.429907     0.610763  0.646016
               10  0.570093     0.529065  0.815652
   equal-size  3   0.542056     0.664381  0.692070
               5   0.542056     0.629125  0.749457
               10  0.542056     0.638374  0.686873
10 equal-width 3   0.383178     0.618840  0.638728
               5   0.588785     0.500753  0.840560
               10  0.607477     0.520606  0.820311
   equal-size  3   0.542056     0.694005  0.654757
               5   0.542056     0.656426  0.672517
               10  0.542056     0.659044  0.663924

In [5]:
train_labels = glass_train_df["CLASS"]
tree_model.fit(glass_train_df,min_samples_split=1)
predictions = tree_model.predict(glass_train_df)
print("Accuracy on training set: {0:.2f}".format(accuracy(predictions,train_labels)))
print("AUC on training set: {0:.2f}".format(auc(predictions,train_labels)))
print("Brier score on training set: {0:.2f}".format(brier_score(predictions,train_labels)))

Accuracy on training set: 0.64
AUC on training set: 0.85
Brier score on training set: 0.41


### Comment on assumptions, things that do not work properly, etc.
The code is not returning the exactly correct values. In order to find the problem, the first thing that
we checked was the correctness of variables: features and classprior. Later, we checked the
correctness of the information_content function; we double checked calculating the information
residual for Mg by hand and also double checked using the dataframe on slide 9 from Decision
Tree lecture and it returned the correct values.

Later on, as we thought to define the tree according to levels which is a very common
representation - specifically, some of group members were taught so in other courses – we
checked the correctness of the predict function, running the code on a smaller glass_test dataset.
The function was able to access to all the nodes and move through the tree correctly. Finally, we
checked the function divide_and_conquer against the slide 3 from Decision Tree lecture. The
three stopping criteria (first three if conditions) are equivalent to the algorithm in slides and we also
discussed with you their correctness. Later on, we checked how the sub-dataframe I_i for each
value v_i for the splitting feature were generated and it worked as expected.

When debugging, it was also tried to number the nodes uniquely using either a global nodeno
variable or letting divide_and_conquer returning current_nodeno and this was very difficult for us to
think as we were so confident with a level-depth representation.

Moreover, as discussed with you on Wednesday 28 th of November, the “when observing
categorical feature values in the test set that were not observed in the training set” problem was
handled correctly in the create_bins function.

Given all this, we think the mistake could be in a stopping criterion or eventually in the
divide_and_conquer function. However, we think that the mistake is not connected to the tree
representation but rather to a misunderstanding of the divide_and_conquer algorithm.
Furthermore, we also think this error could be a minor error as the scores are all plausible values
as well as the training-testing times.

## 2. Define the class DecisionForest

In [9]:
# Define the class DecisionForest with three functions __init__, fit and predict (after the comments):
#
# Input to __init__: 
# self: the object itself
#
# Output from __init__:
# nothing
# 
# This function does not return anything but just initializes the following attributes of the object (self) to None:
# binning, imputatiom, labels, model
#
# Input to fit:
# self: the object itself
# df: a dataframe (where the column names "CLASS" and "ID" have special meaning)
# nobins: no. of bins (default = 10)
# bintype: either "equal-width" (default) or "equal-size"
# min_samples_split: no. of instances required to allow a split (default = 5)
# random_features: no. of features to evaluate at each split (default = 2), 0 means all features (no random sampling)
# notrees: no. of trees in the forest (default = 10)
#
# Output from fit:
# nothing
#
# The result of applying this function should be:
#
# self.binning should be a discretization mapping (see Assignment 1) from df
# self.imputation should be an imputation mapping (see Assignment 1) from df
# self.labels should be the categories of the "CLASS" column of df, set to be of type "category" 
# self.model should be a random forest (for details, see lecture slides)
# Note that the function does not return anything but just assigns values to the attributes of the object.
#
# Hint 1: Redefine divide_and_conquer to take one additional argument; random_features, and instead of
#         evaluating all features choose a random subset, e.g., by np.random.choice (without replacement)
# Hint 2: Generate each tree in the forest from a bootstrap replicate of df, e.g., by np.random.choice 
#         (with replacement) from the index values of df.
#
# Input to predict:
# self: the object itself
# df: a dataframe
# 
# Output from predict:
# predictions: a dataframe with class labels as column names and the rows corresponding to
#              predictions with estimated class probabilities for each row in df, where the class probabilities
#              are the mean of all relative class frequencies in the leaves of the forest into which the instances in
#              df fall
#
# Hint 1: Drop any "CLASS" and "ID" columns first and then apply imputation and binning
# Hint 2: Iterate over the rows calling some sub-function, e.g., make_prediction(row), which for a test row
#         finds all leaf nodes and calculates the average of their class probabilities



In [18]:
glass_train_df = pd.read_csv("glass_train.txt")

glass_test_df = pd.read_csv("glass_test.txt")

forest_model = DecisionForest()

test_labels = glass_test_df["CLASS"]

min_samples_split_values = [1,2,5]
random_features_values = [1,2,5]

parameters = [(min_samples_split,random_features) for min_samples_split in min_samples_split_values 
              for random_features in random_features_values]

results = np.empty((len(parameters),3))

for i in range(len(parameters)):
    t0 = time.perf_counter()
    forest_model.fit(glass_train_df,min_samples_split=parameters[i][0],random_features=parameters[i][1])
    print("Training time {0}: {1:.2f} s.".format(parameters[i],time.perf_counter()-t0))
    t0 = time.perf_counter()
    predictions = forest_model.predict(glass_test_df)
    print("Testing time {0}: {1:.2f} s.".format(parameters[i],time.perf_counter()-t0))
    results[i] = [accuracy(predictions,test_labels),brier_score(predictions,test_labels),
                  auc(predictions,test_labels)] # Assuming that you have defined auc - remove otherwise

results = pd.DataFrame(results,index=pd.MultiIndex.from_product([min_samples_split_values,random_features_values]),
                       columns=["Accuracy","Brier score","AUC"])

results

Training time (1, 1): 7.51 s.
Testing time (1, 1): 0.09 s.
Training time (1, 2): 10.65 s.
Testing time (1, 2): 0.09 s.
Training time (1, 5): 19.78 s.
Testing time (1, 5): 0.09 s.
Training time (2, 1): 7.97 s.
Testing time (2, 1): 0.09 s.
Training time (2, 2): 12.01 s.
Testing time (2, 2): 0.10 s.
Training time (2, 5): 19.42 s.
Testing time (2, 5): 0.10 s.
Training time (5, 1): 4.57 s.
Testing time (5, 1): 0.10 s.
Training time (5, 2): 7.31 s.
Testing time (5, 2): 0.09 s.
Training time (5, 5): 9.75 s.
Testing time (5, 5): 0.09 s.


Accuracy  Brier score       AUC
1 1  0.644860     0.477912  0.860736
  2  0.710280     0.411981  0.892842
  5  0.710280     0.421511  0.895716
2 1  0.663551     0.452421  0.872447
  2  0.626168     0.411684  0.911137
  5  0.654206     0.435847  0.881621
5 1  0.644860     0.457859  0.865636
  2  0.719626     0.410616  0.903016
  5  0.616822     0.455782  0.883897

In [24]:
train_labels = glass_train_df["CLASS"]
forest_model.fit(glass_train_df,min_samples_split=1)
predictions = forest_model.predict(glass_train_df)
print("Accuracy on training set: {0:.2f}".format(accuracy(predictions,train_labels)))
print("AUC on training set: {0:.2f}".format(auc(predictions,train_labels)))
print("Brier score on training set: {0:.2f}".format(brier_score(predictions,train_labels)))

Accuracy on training set: 0.96
AUC on training set: 1.00
Brier score on training set: 0.12


### Comment on assumptions, things that do not work properly, etc.